In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor, LinearRegression, ElasticNet
from sklearn.svm import SVR
from explainerdashboard import RegressionExplainer, ClassifierExplainer, ExplainerDashboard

RANDOM_STATE = 123

In [2]:
smt_2020_df = pd.read_excel('data/I11_SMT Incidents 2020.xlsx')

smt_2020_df.columns = smt_2020_df.iloc[11].values
smt_2020_df = smt_2020_df.iloc[12:]
smt_2020_df = smt_2020_df.reset_index(drop=True)


c:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default


In [3]:
smt_2021_df = pd.read_excel('data/I11_SMT Incidents 2021.xlsx')

smt_2021_df.columns = smt_2021_df.iloc[11].values
smt_2021_df = smt_2021_df.iloc[12:]
smt_2021_df = smt_2021_df.reset_index(drop=True)

In [4]:
smt_2022_df = pd.read_excel('data/I11_SMT Incidents 2022.xlsx')

smt_2022_df.columns = smt_2022_df.iloc[11].values
smt_2022_df = smt_2022_df.iloc[12:]
smt_2022_df = smt_2022_df.reset_index(drop=True)

In [5]:
smt_df = pd.concat([smt_2020_df, smt_2021_df, smt_2022_df])
smt_df.reset_index(drop=True,inplace=True)

pd.set_option('display.max_columns', None)

smt_df

,Incident ID,Original Incident Number,Requisition ID,Created Date (UTC+0),Open Incident Type,Current Incident Type,Status,Status Reason,Company,Customer Department,Region,Site Group,Site,Desk Location,Reported Source,Summary,Impact,Open Priority,Current Priority,Assigned Group,Assigned Group Department,In Progess Time (hhh:mi)\n,Actual Duration/Open Time (hhh:mi),Down Time of CI-Unavailability (hhh:mi),Last Modified Date (UTC+0),First Resolved Date (UTC+0),Last Resolved Date (UTC+0),Created Month,Operational Categorization Tier 1,Operational Categorization Tier 2,Operational Categorization Tier 3,Service+,CI+,Product Categorization Tier 1,Product Categorization Tier 2,Product Categorization Tier 3,Product Name,Resolution Categorization Tier 1,Resolution Categorization Tier 2,Resolution Categorization Tier 3,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Resolution Product Name,Primary Center Code,Target Date,Notes,Resolution,Created by CI-Hotline
0,INC000017348507,NaN,NaN,02.01.2020 11:00,Not assigned,User Service Request,Closed,No Further Action Required,AE,RBEM/MSS4-Q,EMEA,ES,MD1,MD1 150P3,Phone,Ampliar fecha de validez de windows#,4-Minor/Localized,Medium,Medium,Iberia4 -Md-,CI/CER-IB,Not Defined,0260:05,Not Specified,31.03.2020 22:04,03.01.2020 07:21,13.01.2020 07:05,JAN 2020,Request,- None -,- None -,CI-CAT ADMINISTRATION |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,User Rights,- None -,Service,Infrastructure Service,Support Service,NaN,965899,,I: Important: I: Do NOT use this service for r...,"Se habilita la cuenta NT, esta se desactivo ce...",Yes
1,INC000017354421,NaN,NaN,04.01.2020 17:52,Not assigned,User Service Request,Closed,No Further Action Required,AE,RBEM/MSS4-Q,EMEA,ES,MD1,Md1 150P3,Phone,Cuenta de NT caducada#,4-Minor/Localized,Low,Medium,Iberia4 -Md-,CI/CER-IB,Not Defined,0205:20,Not Specified,31.03.2020 22:04,05.01.2020 11:48,13.01.2020 07:12,JAN 2020,Request,Modify,- None -,NT USER MODIFY |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,User Rights,- None -,Service,Infrastructure Service,Support Service,NaN,965859,,30.06.2020 Q: User ID (e.g. xyz8fe): A: GSS2M...,Se amplía de nuevo la fecha de validez del usu...,Yes
2,INC000017400389,NaN,NaN,13.01.2020 09:51,Not assigned,User Service Request,Closed,Customer Follow-Up Required,AE,RBEM/LOP1,EMEA,ES,MD1,Md1 155P3,Email,usuario caducado#,4-Minor/Localized,Low,Medium,Iberia4 -Md-,CI/CER-IB,0:4,0022:02,Not Specified,31.03.2020 22:04,13.01.2020 10:39,14.01.2020 07:53,JAN 2020,Request,- None -,- None -,CI-CAT ADMINISTRATION |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,- None -,- None -,Service,Infrastructure Service,Support Service,NaN,965138,,Su solicitud vía e-mail ha sido transferida a ...,"Como ya dijimos al cerrar el ticket, la valide...",Yes
3,INC000017470045,NaN,10065659,23.01.2020 14:49,Not assigned,User Service Request,Closed,No Further Action Required,M,M/PQA1-RBEM,EMEA,ES,MD1,Md1 162P4,Walk In,My Mobile MSD Micro Pilot : Order Service#,4-Minor/Localized,Low,Low,Iberia4 -Md-,CI/CER-IB,170:8,0502:49,Not Specified,19.03.2020 22:00,13.02.2020 13:38,13.02.2020 13:38,JAN 2020,Request,- None -,- None -,MY MOBILE - LOCAL SUPPORT |IBERIA4,NaN,Service,End Device Service,Mobile Device Service,NaN,Request,- None -,- None -,Service,End Device Service,Mobile Device Service,NaN,NaN,,------------------------------------------ // ...,Móvil MD-Y1733 entregado y configurado,Yes
4,INC000017470799,NaN,10061208,23.01.2020 16:51,Not assigned,User Service Request,Closed,No Further Action Required,TT,TT/SSP-ASA,EMEA,ES,MD,Md,Other,My Mobile MSD Micro Pilot : Order Service#,4-Minor/Localized,Low,Low,Iberia4 -Md-,CI/CER-IB,360:45,0475:00,Not Specified,18.03.2020 22:00,12.02.2020 11:51,12.02.2020 11:51,JAN 2020,Request,- None -,- None -,MY MOBILE - LOCAL SUPPORT |IBERIA4,NaN,Service,End Device Service,Mobile Device Service,NaN,Requ

In [6]:
smt_df['Original Incident Number'].isna().all()

smt_df['Original Incident Number'].notna().sum()

2

In [7]:
output = smt_df.isnull().sum()
output.to_csv('data/missing_values_output.csv')
output

Incident ID                                     0
Original Incident Number                    11947
Requisition ID                               8510
Created Date (UTC+0)                            0
Open Incident Type                              0
Current Incident Type                           0
Status                                          0
Status Reason                                   0
Company                                         0
Customer Department                            68
Region                                          0
Site Group                                     60
Site                                           63
Desk Location                                  70
Reported Source                                 0
Summary                                         0
Impact                                          0
Open Priority                                   0
Current Priority                                0
Assigned Group                                  0


In [8]:
smt_df_transformed = smt_df.copy()

smt_df_transformed['Resolution Time'] = smt_df_transformed['Actual Duration/Open Time (hhh:mi)'].apply(lambda x: round(pd.to_numeric(x.split(':')[0]) + pd.to_numeric(x.split(':')[1]) / 60, 2))

smt_df_transformed

,Incident ID,Original Incident Number,Requisition ID,Created Date (UTC+0),Open Incident Type,Current Incident Type,Status,Status Reason,Company,Customer Department,Region,Site Group,Site,Desk Location,Reported Source,Summary,Impact,Open Priority,Current Priority,Assigned Group,Assigned Group Department,In Progess Time (hhh:mi)\n,Actual Duration/Open Time (hhh:mi),Down Time of CI-Unavailability (hhh:mi),Last Modified Date (UTC+0),First Resolved Date (UTC+0),Last Resolved Date (UTC+0),Created Month,Operational Categorization Tier 1,Operational Categorization Tier 2,Operational Categorization Tier 3,Service+,CI+,Product Categorization Tier 1,Product Categorization Tier 2,Product Categorization Tier 3,Product Name,Resolution Categorization Tier 1,Resolution Categorization Tier 2,Resolution Categorization Tier 3,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Resolution Product Name,Primary Center Code,Target Date,Notes,Resolution,Created by CI-Hotline,Resolution Time
0,INC000017348507,NaN,NaN,02.01.2020 11:00,Not assigned,User Service Request,Closed,No Further Action Required,AE,RBEM/MSS4-Q,EMEA,ES,MD1,MD1 150P3,Phone,Ampliar fecha de validez de windows#,4-Minor/Localized,Medium,Medium,Iberia4 -Md-,CI/CER-IB,Not Defined,0260:05,Not Specified,31.03.2020 22:04,03.01.2020 07:21,13.01.2020 07:05,JAN 2020,Request,- None -,- None -,CI-CAT ADMINISTRATION |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,User Rights,- None -,Service,Infrastructure Service,Support Service,NaN,965899,,I: Important: I: Do NOT use this service for r...,"Se habilita la cuenta NT, esta se desactivo ce...",Yes,260.08
1,INC000017354421,NaN,NaN,04.01.2020 17:52,Not assigned,User Service Request,Closed,No Further Action Required,AE,RBEM/MSS4-Q,EMEA,ES,MD1,Md1 150P3,Phone,Cuenta de NT caducada#,4-Minor/Localized,Low,Medium,Iberia4 -Md-,CI/CER-IB,Not Defined,0205:20,Not Specified,31.03.2020 22:04,05.01.2020 11:48,13.01.2020 07:12,JAN 2020,Request,Modify,- None -,NT USER MODIFY |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,User Rights,- None -,Service,Infrastructure Service,Support Service,NaN,965859,,30.06.2020 Q: User ID (e.g. xyz8fe): A: GSS2M...,Se amplía de nuevo la fecha de validez del usu...,Yes,205.33
2,INC000017400389,NaN,NaN,13.01.2020 09:51,Not assigned,User Service Request,Closed,Customer Follow-Up Required,AE,RBEM/LOP1,EMEA,ES,MD1,Md1 155P3,Email,usuario caducado#,4-Minor/Localized,Low,Medium,Iberia4 -Md-,CI/CER-IB,0:4,0022:02,Not Specified,31.03.2020 22:04,13.01.2020 10:39,14.01.2020 07:53,JAN 2020,Request,- None -,- None -,CI-CAT ADMINISTRATION |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,- None -,- None -,Service,Infrastructure Service,Support Service,NaN,965138,,Su solicitud vía e-mail ha sido transferida a ...,"Como ya dijimos al cerrar el ticket, la valide...",Yes,22.03
3,INC000017470045,NaN,10065659,23.01.2020 14:49,Not assigned,User Service Request,Closed,No Further Action Required,M,M/PQA1-RBEM,EMEA,ES,MD1,Md1 162P4,Walk In,My Mobile MSD Micro Pilot : Order Service#,4-Minor/Localized,Low,Low,Iberia4 -Md-,CI/CER-IB,170:8,0502:49,Not Specified,19.03.2020 22:00,13.02.2020 13:38,13.02.2020 13:38,JAN 2020,Request,- None -,- None -,MY MOBILE - LOCAL SUPPORT |IBERIA4,NaN,Service,End Device Service,Mobile Device Service,NaN,Request,- None -,- None -,Service,End Device Service,Mobile Device Service,NaN,NaN,,------------------------------------------ // ...,Móvil MD-Y1733 entregado y configurado,Yes,502.82
4,INC000017470799,NaN,10061208,23.01.2020 16:51,Not assigned,User Service Request,Closed,No Further Action Required,TT,TT/SSP-ASA,EMEA,ES,MD,Md,Other,My Mobile MSD Micro Pilot : Order Service#,4-Minor/Localized,Low,Low,Iberia4 -Md-,CI/CER-IB,360:45,0475:00,Not Specified,18.03.2020 22:00,12.02.2020 11:51,12.02.2020 11:51,JAN 2020,Request,- None -,- None -,MY MOBILE - LOCAL SUPPORT |IBERIA4,NaN,Service,End De

In [9]:
ocurrencias = smt_df_transformed['Site Group'].value_counts()
print(len(ocurrencias))
print(ocurrencias)

16
ES    11809
DE       36
PT       13
MX        9
CZ        3
US        3
RO        3
DK        2
RS        2
VN        2
MA        2
FR        1
CN        1
TH        1
BR        1
SG        1
Name: Site Group, dtype: int64


In [10]:
#Posibles a borrar ['Current Incident Type', 'Region', 'Impact', 'Site','Product Categorization Tier 1', 'Resolution Categorization Tier 1', 'Resolution Categorization Tier 2', ]
#Posibles a no borrar ['Customer Department', 'Primary Center Code']

smt_df_clean = smt_df_transformed.drop(['Original Incident Number', 'Requisition ID', 'Created Date (UTC+0)', 'Status', 'Status Reason', 'Customer Department', 'Summary', 'Desk Location', 'Impact', 'In Progess Time (hhh:mi)\n', 'Actual Duration/Open Time (hhh:mi)',
                                         'Down Time of CI-Unavailability (hhh:mi)', 'Last Modified Date (UTC+0)', 'First Resolved Date (UTC+0)', 'Last Resolved Date (UTC+0)', 'Created Month', 'Service+', 'CI+', 'Product Name', 
                                         'Resolution Categorization Tier 3', 'Resolution Product Name', 'Primary Center Code', 'Target Date', 'Notes', 'Resolution'], axis=1)
smt_df_clean.dropna(inplace=True)
smt_df_clean

,Incident ID,Open Incident Type,Current Incident Type,Company,Region,Site Group,Site,Reported Source,Open Priority,Current Priority,Assigned Group,Assigned Group Department,Operational Categorization Tier 1,Operational Categorization Tier 2,Operational Categorization Tier 3,Product Categorization Tier 1,Product Categorization Tier 2,Product Categorization Tier 3,Resolution Categorization Tier 1,Resolution Categorization Tier 2,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Created by CI-Hotline,Resolution Time
0,INC000017348507,Not assigned,User Service Request,AE,EMEA,ES,MD1,Phone,Medium,Medium,Iberia4 -Md-,CI/CER-IB,Request,- None -,- None -,Service,Infrastructure Service,Support Service,Request,User Rights,Service,Infrastructure Service,Support Service,Yes,260.08
1,INC000017354421,Not assigned,User Service Request,AE,EMEA,ES,MD1,Phone,Low,Medium,Iberia4 -Md-,CI/CER-IB,Request,Modify,- None -,Service,Infrastructure Service,Support Service,Request,User Rights,Service,Infrastructure Service,Support Service,Yes,205.33
2,INC000017400389,Not assigned,User Service Request,AE,EMEA,ES,MD1,Email,Low,Medium,Iberia4 -Md-,CI/CER-IB,Request,- None -,- None -,Service,Infrastructure Service,Support Service,Request,- None -,Service,Infrastructure Service,Support Service,Yes,22.03
3,INC000017470045,Not assigned,User Service Request,M,EMEA,ES,MD1,Walk In,Low,Low,Iberia4 -Md-,CI/CER-IB,Request,- None -,- None -,Service,End Device Service,Mobile Device Service,Request,- None -,Service,End Device Service,Mobile Device Service,Yes,502.82
4,INC000017470799,Not assigned,User Service Request,TT,EMEA,ES,MD,Other,Low,Low,Iberia4 -Md-,CI/CER-IB,Request,- None -,- None -,Service,End Device Service,Mobile Device Service,Request,- None -,Service,End Device Service,Mobile Device Service,Yes,475.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11944,INC000023672229,User Service Restoration,User Service Restoration,C,EMEA,ES,MD,Phone,Low,Low,Spain1,BD/PFE-IA,Failure,Application,- None -,Service,Application Service,Application Service,Handling,Unclear,Service,Application Service,Application Service,Yes,23.93
11945,INC000023674299,User Service Restoration,User Service Restoration,GS,EMEA,ES,MD,Phone,Low,Low,Spain1,BD/PFE-IA,Failure,- None -,- None -,Service,IT Service,IT Service,Handling,Unclear,Service,IT Service,IT Service,Yes,0.58
11946,INC000023674336,User Service Request,User Service Request,AE,EMEA,ES,MD1,Walk In,Low,Low,Spain1,BD/PFE-IA,Request,- None -,- None -,Service,IT Service,IT Service,Request,- None -,Service,IT Service,IT Service,No,0.02
11947,INC000023678629,User Service Request,User Service Request,AE,EMEA,ES,MD1,Phone,Low,Low,Spain1,BD/PFE-IA,Request,- None -,- None -,Service,Infrastructure Service,Support Service,Request,User Rights,Service,Infrastructure Service,Support Service,Yes,0.27


In [11]:
columns_list = smt_df_clean.columns
columns_list = [elem for elem in columns_list if elem not in ['Incident ID', 'Resolution Time']] 
smt_df_one_hot = pd.get_dummies(smt_df_clean, columns=columns_list)
smt_df_one_hot

,Incident ID,Resolution Time,Open Incident Type_Infrastructure Event,Open Incident Type_Infrastructure Restoration,Open Incident Type_Not assigned,Open Incident Type_User Service Request,Open Incident Type_User Service Restoration,Current Incident Type_Infrastructure Restoration,Current Incident Type_User Service Request,Current Incident Type_User Service Restoration,Company_AA,Company_AE,Company_ATMO,Company_BCI,Company_BD,Company_BT,Company_C,Company_CC,Company_CI,Company_CP,Company_CR,Company_CS,Company_DA,Company_DC,Company_EB,Company_EU2,Company_EXTERN-UNKNOWN,Company_GR,Company_GS,Company_IOX,Company_M,Company_PA,Company_PC,Company_PS,Company_PT,Company_RBEI,Company_RBIB,Company_RBIT,Company_RBNA,Company_RN,Company_SO,Company_ST,Company_SX,Company_TT,Company_VW,Company_XC,Region_AMERICAS,Region_APAC,Region_EMEA,Site Group_BR,Site Group_CN,Site Group_CZ,Site Group_DE,Site Group_DK,Site Group_ES,Site Group_FR,Site Group_MA,Site Group_MX,Site Group_PT,Site Group_RO,Site Group_RS,Site Group_SG,Site Group_TH,Site Group_US,Site Group_VN,Site_AD,Site_AI,Site_AJ,Site_ALD,Site_BAC,Site_BG,Site_BJ,Site_BL,Site_BUK,Site_CA,Site_CAS,Site_CLJ,Site_CS,Site_CSC,Site_FE,Site_FE66,Site_HA3,Site_HC1,Site_HI,Site_HMJ,Site_JU2,Site_JU3,Site_KA,Site_LE,Site_LIC,Site_LIS,Site_MD,Site_MD1,Site_MD3,Site_MX2,Site_RNG,Site_RT,Site_SGM,Site_SGP,Site_SH,Site_SLP,Site_SPM,Site_SSE,Site_SZH,Site_TL,Site_VG,Site_VG2,Site_VG3,Site_WA1,Site_WE,Reported Source_Chat,Reported Source_Direct Input,Reported Source_Email,Reported Source_Not Defined,Reported Source_Other,Reported Source_Phone,Reported Source_Self Service,Reported Source_Walk In,Reported Source_Web,Open Priority_Critical,Open Priority_High,Open Priority_Low,Open Priority_Medium,Current Priority_High,Current Priority_Low,Current Priority_Medium,Assigned Group_Iberia4,Assigned Group_Iberia4 - EXT,Assigned Group_Iberia4 -Md-,Assigned Group_Spain1,Assigned Group_Spain1 - EXT,Assigned Group Department_BD/PFE-IA,Assigned Group Department_CI/CER-IB,Operational Categorization Tier 1_Failure,Operational Categorization Tier 1_Request,Operational Categorization Tier 2_- None -,Operational Categorization Tier 2_Add,Operational Categorization Tier 2_Application,Operational Categorization Tier 2_Consult,Operational Categorization Tier 2_Delete,Operational Categorization Tier 2_Install,Operational Categorization Tier 2_Installation,Operational Categorization Tier 2_Modify,Operational Categorization Tier 2_Patch,Operational Categorization Tier 2_Repair,Operational Categorization Tier 2_Reset,Operational Categorization Tier 2_Unlock,Operational Categorization Tier 3_- None -,Operational Categorization Tier 3_Password,Operational Categorization Tier 3_Rights,Operational Categorization Tier 3_Role,Operational Categorization Tier 3_Space,Operational Categorization Tier 3_User,Product Categorization Tier 1_Process,Product Categorization Tier 1_Service,Product Categorization Tier 2_Application Service,Product Categorization Tier 2_Business Service,Product Categorization Tier 2_Commercial Process,Product Categorization Tier 2_Commercial Service,Product Categorization Tier 2_End Device Service,Product Categorization Tier 2_IT Service,Product Categorization Tier 2_Infrastructure Service,Product Categorization Tier 3_Application Service,Product Categorization Tier 3_Business Process,Product Categorization Tier 3_Business Service,Product Categorization Tier 3_Central Mobile APP,Product Categorization Tier 3_Communication Service,Product Categorization Tier 3_Directory Service,Product Categorization Tier 3_Enterprise Application Service,Product Categorization Tier 3_File Service,Product Categorization Tier 3_IT Service,Product Categorization Tier 3_Infrastructure IT Service,Product Categorization Tier 3_Local Software Service,Product Categorization Tier 3_Mobile Device Service,Product Categorization Tier 3_Network Service,Product Categorization Tier 3_Output Management Service,Product Categorization Tier 3_Peacy Package,Pr

In [12]:
X = smt_df_one_hot.drop(['Incident ID', 'Resolution Time'], axis=1).values
y = smt_df_one_hot['Resolution Time'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

In [13]:
gbr = GradientBoostingRegressor(random_state=RANDOM_STATE, n_estimators=600, max_depth=4, learning_rate=0.01, loss='squared_error')

gbr.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
gbr_predictions = gbr.predict(X_test)

mse = mean_squared_error(y_test, gbr_predictions)
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

gbr_predictions

The mean squared error (MSE) on test set: 13864.0491


array([200.15223449,  54.26017805,  13.20002977, ...,  25.38500507,
        15.56102792,  29.83592101])

In [ ]:
y_test

array([7.0300e+00, 1.4488e+02, 2.0000e-02, ..., 1.4800e+00, 2.1420e+01,
       9.4300e+00])

In [ ]:
X_test_df = pd.DataFrame(X_test, columns=smt_df_one_hot.drop(['Incident ID', 'Resolution Time'], axis=1).columns)
y_test_df = pd.DataFrame(y_test, columns=['Last Resolution Time'])

y_test_df

,Last Resolution Time
0,7.03
1,144.88
2,0.02
3,0.30
4,212.85
...,...
2373,0.02
2374,1049.42
2375,1.48
2376,21.42


In [ ]:
df_predictions = pd.DataFrame({'Predicciones': gbr_predictions})

df_predictions

,Predicciones
0,200.152234
1,54.260178
2,13.200030
3,-0.235718
4,24.229564
...,...
2373,22.419527
2374,350.429567
2375,25.385005
2376,15.561028


In [ ]:
explainer = RegressionExplainer(gbr, X_test_df, y_test_df)

ExplainerDashboard(explainer).run()

Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...
Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating shap interaction values...
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.
Calculating dependencies...
Calculating importances...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks.

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_3641a977-059d-4730-84be-d1f50ccc62ba (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000245A5C2C2E0>: Failed to establish a new connection: [WinError 10049] La dirección solicitada no es válida en este contexto'))

In [ ]:
"""X = combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).values
y = combined_df_one_hot['Last Resolution Time'].values
"""

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

sgd = SGDRegressor(random_state=RANDOM_STATE)

sgd.fit(X_train, y_train)

sgd_predictions = sgd.predict(X_test)

mse = mean_squared_error(y_test, sgd_predictions)
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

sgd_predictions

The mean squared error (MSE) on test set: 3801.7150


array([ 60.91659446,  95.08133065, -88.10622814,  -3.88085863,
        41.85841488, 141.24441651, -26.57587466,  20.90081538,
        10.56533831,  17.81121568,   2.63001958, 120.96857605,
        26.66536556, -12.06130336,  10.56533831,   7.52218458,
        81.13035228,  35.69121395, -12.21620214,  36.04454114,
        28.66417133,  13.40435903,  15.45750331, -17.86335008,
        83.09520595,  46.77668498,  31.15237614,   5.3465526 ,
         2.63001958,  -3.32283821,  41.30258264, 148.40459087,
        -3.32283821,  90.25909979,  -4.36101115, -16.10789529,
        16.78825011,   1.81863393, -19.42276862,  20.90081538,
       -14.92169854,  75.93722625,   9.54827851, 151.63267879,
       103.54809535,  10.58220671,  31.47605944,  10.56533831,
        85.72540983,  98.92289764,  26.04932775,  36.04454114,
       138.12180535,  52.39647573,  36.04454114,  33.63718919,
         3.69840898,   4.82928651,   3.72307004,  40.44527946,
       105.22150551, -15.68478017, 166.47413371,  -0.32

In [ ]:
X_test_df = pd.DataFrame(X_test, columns=combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).columns)
y_test_df = pd.DataFrame(y_test, columns=['Last Resolution Time'])

y_test_df

,Last Resolution Time
0,1.971111
1,64.589722
2,88.772222
3,0.000278
4,37.225556
...,...
89,0.010278
90,0.000556
91,116.825278
92,72.316389


In [ ]:
"""len(combined_df_one_hot.index) * 0.2 # 93.4 trunca a 94
first_test_index = len(combined_df_one_hot.index) - 94
X_test_df = combined_df_one_hot.iloc[first_test_index:].drop(['Incident Number', 'Last Resolution Time'], axis=1)
y_test_df = combined_df_one_hot.iloc[first_test_index:][['Last Resolution Time']]"""

"len(combined_df_one_hot.index) * 0.2 # 93.4 trunca a 94\nfirst_test_index = len(combined_df_one_hot.index) - 94\nX_test_df = combined_df_one_hot.iloc[first_test_index:].drop(['Incident Number', 'Last Resolution Time'], axis=1)\ny_test_df = combined_df_one_hot.iloc[first_test_index:][['Last Resolution Time']]"

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

svr = SVR()

svr.fit(X_train, y_train)

svr_predictions = svr.predict(X_test)

mse = mean_squared_error(y_test, svr_predictions)
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

svr_predictions

The mean squared error (MSE) on test set: 4757.2184


array([ 5.70670165e+00,  5.51075283e+00,  4.51085722e+00, -8.49714361e-02,
        3.77876128e+00,  5.69165457e+00, -8.62187662e-02,  4.57039242e-01,
       -7.06793422e-02,  1.68041804e+00,  1.00616278e-01,  2.22696857e+00,
        3.61041951e+00,  2.10975670e-02, -7.06793422e-02,  1.00788792e-01,
        2.64560428e+00,  1.38500517e+00, -8.16851016e-02,  6.56031411e+00,
        5.86154621e-01,  1.01047512e-01,  1.17525153e-01, -7.89184154e-02,
        6.59562616e+00,  5.45759883e+00,  5.23406409e+00,  3.47928114e-02,
        1.00616278e-01,  1.01083115e-01,  1.55996154e+00,  4.91465410e+00,
        1.01083115e-01,  5.62237176e+00, -3.37281552e-02,  6.39619548e-03,
        1.70201969e-02, -8.52151780e-03,  4.46474385e+00,  4.57039242e-01,
        2.61132484e-02,  4.61342115e+00,  1.01425339e-01,  6.34715406e+00,
        5.28805162e+00,  3.28702600e-01,  5.00012805e+00, -7.06793422e-02,
        6.18386521e+00,  6.32952292e+00,  1.09231828e-01,  6.56031411e+00,
        6.68470945e+00,  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

linReg = LinearRegression()

linReg.fit(X_train, y_train)

linReg_predictions = linReg.predict(X_test)

mse = mean_squared_error(y_test, linReg_predictions)
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

linReg_predictions

In [ ]:
explainer = RegressionExplainer(gbr, X_test_df, y_test_df)

ExplainerDashboard(explainer).run()

In [ ]:
explainer = RegressionExplainer(sgd, X_test_df, y_test_df)

ExplainerDashboard(explainer).run()

In [ ]:
pd.reset_option('display.max_columns')